##### 将NII文件转换为PNG

In [ ]:
import nibabel as nib
import os
import numpy as np
# 转换成图像
import imageio
imgfile="temp/0017691773FILT.nii"
file_path="/mnt/hdd1/zhulu/hospital/first_stage_nii/zhao_yong_xing_20180823_005_B01-HX-cognitive_disorde_Ax_SWAN_new.nii"
img = nib.load(file_path)
img_fdata = img.get_fdata()

    
(x, y, z) = img.shape
# z是图像的序列
output_dir = '/mnt/hdd1/zhulu/hospital/first_stage_nii/swan'
for i in range(z):
    # 选择哪个方向的切片都可以
    silce = img_fdata[:, :, i]
    # 保存图像
    # 归一化切片数据到0-255范围，并转换为uint8类型
    slice_data = ((silce - np.min(silce)) / (np.max(silce) - np.min(silce)) * 255).astype(np.uint8)

    # 将当前切片保存为 PNG
    output_file = os.path.join(output_dir, '{}.png'.format(i+1))
    imageio.imwrite(output_file, slice_data)
                            
x=1+3

##### 将病人数据txt按照高/低分辨率进行归类，合成总的数据集，并删除掉其中非出血点标签，清除空文件

In [1]:
import os
import shutil

def clear_folder(folder_path):
    # 检查文件夹是否存在
    if os.path.exists(folder_path):
        # 遍历文件夹中的所有文件和子文件夹
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                # 删除文件或文件夹
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)  # 删除文件或符号链接
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # 删除文件夹及其内容
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        print(f"Folder {folder_path} does not exist.")

# # 清理高分辨率和低分辨率文件夹
# clear_folder(high_resolution_folder)
# clear_folder(low_resolution_folder)

# print("Folders have been cleared.")


In [3]:
import os 
import shutil
#此处训练集和验证集都要进行删除
#训练集如果不删除其他标签数据报错明显，但是测试集不删除报错提示不明显
""" 注意是train和val都要删除 """
# train_txt="/mnt/hdd1/zhulu/hospital/labels/train"  
# txt_files=os.listdir(train_txt)

def adjust_yolo_file(file_dir):
    txt_files=os.listdir(file_dir)
    for file in txt_files:
        file_path=os.path.join(file_dir,file)
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # 选出符合出血点标签的行
        updated_lines = [line for line in lines if line.strip().split()[0] == "0"]
        
        if len(updated_lines)==0: #如果文件没有出血点的标注，则直接删除文件
            os.remove(file_path)
        else:
            # 将更新后的内容写回文件
            with open(file_path, 'w') as f:
                f.writelines(updated_lines)
                
low_resolution=["Patient-0000406857","Patient-0000589446",'Patient-0000745885','Patient-0003537968'
                ,'Patient-0010134951','Patient-0018227242','Patient-0018463963']
all_second_patient_path="/mnt/hdd1/zhulu/blood_stage2/blood_anno/PNG"
all_second_patients=os.listdir(all_second_patient_path)

high_resolution_folder="/mnt/hdd1/zhulu/hospital/high_resolution"
low_resolution_folder="/mnt/hdd1/zhulu/hospital/low_resolution"

clear_folder(high_resolution_folder)
clear_folder(low_resolution_folder)

#移动文件，先移动label
for patient in all_second_patients:
    patient_lable_path=os.path.join(all_second_patient_path,patient,"label")
    if patient in low_resolution:
        for item in os.listdir(patient_lable_path):
            source_item = os.path.join(patient_lable_path, item)
            # 因为源文件夹下的文件命名没有病人编号，因此需要重命名过去
            target_item = os.path.join(low_resolution_folder, patient+"_"+item)
            # target_item = os.path.join(low_resolution_folder, item)
            if os.path.isfile(source_item):
                shutil.copy2(source_item, target_item)
    else:
        for item in os.listdir(patient_lable_path):
            source_item = os.path.join(patient_lable_path, item)
            target_item = os.path.join(high_resolution_folder,patient+"_"+item)
            if os.path.isfile(source_item):
                shutil.copy2(source_item, target_item)

#删除非出血点标签，删除空白文件txt
adjust_yolo_file(high_resolution_folder)
adjust_yolo_file(low_resolution_folder)


#### 按照病人编号进行排序，计算出血点个数，按照8：2进行训练集和测试集的划分

In [1]:
""" 以下复制移动文件是通过手动的方式,xftp文件可以按照文件名进行排序 """
import os
import re

def change_seq(file_dir):
    train_labels=os.listdir(file_dir)
    for train_label in train_labels:
        patien_name,suffix=train_label.split("_")
        img,seq,layer=suffix.split("-")
        new_file_name=patien_name+"_img-00005-"+layer
        old_file=os.path.join(file_dir,train_label)
        new_file=os.path.join(file_dir,new_file_name)
        os.rename(old_file, new_file)

def extract_patient_id(file_name):
    match = re.search(r'Patient-(\d+)', file_name)
    if match:
        return int(match.group(1))
    return None

high_folder="/mnt/hdd1/zhulu/hospital/high_resolution"

change_seq(high_folder)  # 先修改每个文件的序列号，swi为00005
high_files=os.listdir(high_folder)
# 按照病人编号进行排序，确保同一个病人只在训练集或者测试集
sorted_file_list = sorted(high_files, key=extract_patient_id)  
all_CMB_annos=0
for file in sorted_file_list:
    with open(os.path.join(high_folder,file), 'r') as f:
        lines = f.readlines()
    all_CMB_annos+=len(lines)  
    #Patient-0018471528_img-00005-00059.txt 高分辨率下，8：2的分界点
    #后面是服务器上手动复制粘贴过去的（服务器上本身已经按照病患编号进行排序了）
    if  all_CMB_annos==540:  #540是总的all_CMB_annos，按照8：2的划分中间
        print(file)

Patient-0018471528_img-00005-00059.txt


In [11]:
# 改变label的编码，swi为00005,phase为00500
import os
train_label_dir="/mnt/hdd1/zhulu/hospital/labels/high_resolution/val"
train_labels=os.listdir(train_label_dir)

def change_seq(file_dir):
    train_labels=os.listdir(file_dir)
    for train_label in train_labels:
        patien_name,suffix=train_label.split("_")
        img,seq,layer=suffix.split("-")
        new_file_name=patien_name+"_img-00005-"+layer
        old_file=os.path.join(file_dir,train_label)
        new_file=os.path.join(file_dir,new_file_name)
        os.rename(old_file, new_file)
    

##### 根据label txt文件名，将对应的图像也划分到images下面的训练和测试文件中

In [13]:
import os
import shutil
""" 注意train和val都需要换 """
split="val"
train_lable_files_dir=f"/mnt/hdd1/zhulu/hospital/labels/high_resolution/{split}"
second_patients_dir="/mnt/hdd1/zhulu/blood_stage2/blood_anno/PNG"
target_dir=f"/mnt/hdd1/zhulu/hospital/images/high_resolution/{split}"
patients=os.listdir(second_patients_dir)
train_label_files=os.listdir(train_lable_files_dir)
for train_label_file in train_label_files:
    patient_id=train_label_file.split("_")[0]
    img_id=train_label_file.split("_")[1].split(".")[0]+".png"
    patient_img_dir=os.path.join(second_patients_dir,patient_id,"Series-Ax SWAN new")
    img_path=os.path.join(patient_img_dir,img_id)
    # 将原本文件patient_id_img_id.png整个名称也要复制过去
    target_path=os.path.join(target_dir,train_label_file.split(".")[0]+".png") 
    shutil.copy2(img_path, target_path)
    
    

##### 第二阶段数据进行8:2划分

In [ ]:
import os
import random
import shutil
from math import ceil

# 定义源文件夹和目标文件夹
resolution="high"
is_CMB="CMB"
source_dir = f'/mnt/hdd1/zhulu/hospital/second_stage/train_model/{resolution}/all/{is_CMB}'
train_dir = f'/mnt/hdd1/zhulu/hospital/second_stage/train_model/{resolution}/train/{is_CMB}'
test_dir = f'/mnt/hdd1/zhulu/hospital/second_stage/train_model/{resolution}/test/{is_CMB}'

# 创建目标文件夹，如果它们不存在
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# 获取源文件夹中的所有文件
all_files = os.listdir(source_dir)

# 设置随机种子以确保每次运行结果一致
random.seed(42)

# 打乱文件列表
random.shuffle(all_files)

# 计算80%和20%的切分点
split_point = ceil(len(all_files) * 0.8)

# 切分文件列表
train_files = all_files[:split_point]
test_files = all_files[split_point:]

# 将文件复制到训练和测试文件夹
for file in train_files:
    shutil.copy(os.path.join(source_dir, file), train_dir)

for file in test_files:
    shutil.copy(os.path.join(source_dir, file), test_dir)

print(f"已将{len(train_files)}个文件复制到{train_dir}")
print(f"已将{len(test_files)}个文件复制到{test_dir}")


### 将.dcm数据转换为nii数据

In [ ]:
import pydicom
import nibabel as nib
import numpy as np
import os

def dcm_to_nii(dcm_folder, nii_filename):
    """
    将一个包含多个DICOM文件的文件夹转换为NIfTI格式文件。
    
    参数：
    dcm_folder : str : DICOM文件所在的文件夹路径
    nii_filename : str : 输出NIfTI文件的路径
    """
    # 获取所有的DICOM文件
    dcm_files = [os.path.join(dcm_folder, f) for f in os.listdir(dcm_folder) if f.endswith('.dcm')]
    
    # 按照DICOM文件的InstanceNumber或者SliceLocation排序（确保切片顺序正确）
    dcm_files.sort(key=lambda x: int(pydicom.dcmread(x).InstanceNumber))
    
    # 读取所有DICOM切片
    slices = []
    for dcm_file in dcm_files:
        dicom_data = pydicom.dcmread(dcm_file)
        slices.append(dicom_data.pixel_array)
    
    # 将切片数据转换为3D NumPy数组
    image_3d = np.stack(slices, axis=-1)
    
    # 获取DICOM的相关元数据（如空间维度、像素间距等）
    first_dcm = pydicom.dcmread(dcm_files[0])
    affine = np.diag([first_dcm.PixelSpacing[0], first_dcm.PixelSpacing[1], first_dcm.SliceThickness, 1])  # 假设Affine为标准

    # 创建NIfTI图像
    nii_img = nib.Nifti1Image(image_3d, affine)
    
    # 保存NIfTI文件
    nib.save(nii_img, nii_filename)
    print(f"成功将DICOM转换为NIfTI格式：{nii_filename}")

# 使用示例
dcm_folder = '/mnt/hdd1/zhulu/005_AxSWANnew'  # 你的DICOM文件夹路径
nii_filename = '/mnt/hdd1/zhulu/output_file.nii'  # 输出NIfTI文件路径
dcm_to_nii(dcm_folder, nii_filename)


In [ ]:
import os
file_folders="/mnt/hdd1/zhulu/blood_stage2/blood_anno2/new3"
file_names=os.listdir(file_folders)
for file_name in file_names:
    file_paths=os.path.join(file_folders,file_name)
    files_path=os.listdir(file_paths)
    for file_path in files_path:
        file_path=os.path.join(file_paths,file_path)
        nii_filename=os.path.join(file_paths,f"{file_name}.nii")
        dcm_to_nii(dcm_folder, nii_filename)
        